In [1]:
import pymongo
from pymongo import MongoClient

from pandas import DataFrame, read_csv
import json
import matplotlib.pyplot as plt
import pandas as pd #this is how I usually import pandas
import sys #only needed to determine Python version number
import matplotlib #only needed to determine Matplotlib version number

import ast
import time
from datetime import datetime


### Get db reference

In [2]:
client = MongoClient('mongodb://127.0.0.1:27017')
db = client.sample


In [4]:
db.list_collection_names()


[u'system.indexes',
 u'users_in_teams',
 u'teams_centralities',
 u'users',
 u'likes',
 u'shots_single',
 u'user_ranking',
 u'shots_ranking',
 u'test',
 u'user_translate',
 u'teams',
 u'skills',
 u'shots',
 u'test_copy',
 u'followers',
 u'user_team_first_shot']

In [21]:
db.teams.find()[0]

{u'_id': ObjectId('5bb259ace2850521dc2f1856'),
 u'avatar_url': u'https://cdn.dribbble.com/users/673130/avatars/normal/4fb0be803bf3f04fdf007117e4622bfc.png?1507932552',
 u'bio': u'We are a global team of strategy, design and development experts.',
 u'buckets_count': 0,
 u'buckets_url': u'https://api.dribbble.com/v1/users/673130/buckets',
 u'can_upload_shot': True,
 u'comments_received_count': 0,
 u'created_at': 1413503476,
 u'followers_count': 1042,
 u'followers_url': u'https://api.dribbble.com/v1/users/673130/followers',
 u'following_url': u'https://api.dribbble.com/v1/users/673130/following',
 u'followings_count': 1943,
 u'html_url': u'https://dribbble.com/citrusbyte',
 u'id': 673130,
 u'likes_count': 471,
 u'likes_received_count': 40,
 u'likes_url': u'https://api.dribbble.com/v1/users/673130/likes',
 u'links': {u'twitter': u'https://twitter.com/citrusbyte',
  u'web': u'https://citrusbyte.com'},
 u'location': u'Los Angeles',
 u'members_count': 18,
 u'members_url': u'https://api.dribbb

### Add users DataFrame to db's new collection 'users'

In [ ]:
Location = r'/home/simo/UnInfo/Tesi Magistrale/Dataset/users.tsv'
df = pd.read_csv(Location, sep='\t')
desc = df[df.columns[1]]


In [ ]:
print desc[5]

In [ ]:
len(desc)

#### This function remove some problematic tags 

In [ ]:

def parseUser(user):
    temp = user.replace(": null",": \"null\"")
    
    temp = temp.replace(" \"can_upload_shot\": false,","")
    temp = temp.replace(" \"pro\": false,","")
    temp = temp.replace("\"can_upload_shot\": false,","")
    temp = temp.replace("\"pro\": false,","")
    temp = temp.replace(" \"can_upload_shot\": false","")
    temp = temp.replace(" \"pro\": false","")
    
    temp = temp.replace(" \"can_upload_shot\": true,","")
    temp = temp.replace(" \"pro\": true,","")
    temp = temp.replace("\"can_upload_shot\": true,","")
    temp = temp.replace("\"pro\": true,","")
    temp = temp.replace(" \"can_upload_shot\": true","")
    temp = temp.replace(" \"pro\": true","")
    
    # rimuove la virgola al fondo se esiste
    if temp[len(temp)-2] == ',':
        l = list(temp)
        l[len(temp)-2] = ''
        temp = ''.join(l)
    
    
    return temp
    

# test su campione di user

In [ ]:


db.drop_collection('test_users')

campione = []
#campione.append(desc[12345])
#campione.append(desc[12346])
campione.append(desc[1])


for elem in campione:
    temp = parseUser(elem)
    json_elem = json.loads(temp)

    json_elem.pop('followers_url', None)
    json_elem.pop('following_url', None)
    json_elem.pop('avatar_url', None)
    json_elem.pop('buckets_url', None)
    json_elem.pop('likes_url', None)
    json_elem.pop('links', None)
    json_elem.pop('members_url', None)
    json_elem.pop('projects_url', None)
    json_elem.pop('shots_url', None)
    json_elem.pop('team_shots_url', None)
    json_elem.pop('type', None)
    json_elem.pop('teams_url', None)
    json_elem.pop('html_url', None)
    
    d = datetime.strptime(json_elem['created_at'], '%Y-%m-%dT%H:%M:%SZ')
    unixtime = time.mktime(d.timetuple())
    json_elem['created_at'] = int(unixtime)
    
    d = datetime.strptime(json_elem['updated_at'], '%Y-%m-%dT%H:%M:%SZ')
    unixtime = time.mktime(d.timetuple())
    json_elem['updated_at'] = int(unixtime)
    
    #print json_elem
    
    db.test_users.insert_one(json_elem)
    



In [ ]:


cursor = db.test_users.find()
for c in cursor:
    print c['username'], c['created_at'] , c['updated_at']

In [ ]:

dato =  json.loads('{"teams_count": 0, "shots_count": 0, "id": 1049422}')

# Insert users on mongo db

In [ ]:
%%time

db.drop_collection('users')

for elem in desc:
    temp = parseUser(elem)
    
    try:
        json_elem = json.loads(temp)
    except:
        print temp
        break
        
    json_elem.pop('followers_url', None)
    json_elem.pop('following_url', None)
    json_elem.pop('avatar_url', None)
    json_elem.pop('buckets_url', None)
    json_elem.pop('likes_url', None)
    json_elem.pop('links', None)
    json_elem.pop('members_url', None)
    json_elem.pop('projects_url', None)
    json_elem.pop('shots_url', None)
    json_elem.pop('type', None)
    json_elem.pop('html_url', None)
    
    try:
        
        d = datetime.strptime(json_elem['created_at'], '%Y-%m-%dT%H:%M:%SZ')
        unixtime = time.mktime(d.timetuple())
        json_elem['created_at'] = int(unixtime)

        d = datetime.strptime(json_elem['updated_at'], '%Y-%m-%dT%H:%M:%SZ')
        unixtime = time.mktime(d.timetuple())
        json_elem['updated_at'] = int(unixtime)
    
    except:
        continue
    
    db.users.insert_one(json_elem)
    


In [ ]:
cursor = db.users.find()
print cursor.count()

non tutti i record hanno i campi 'created_at' e 'update_at', quindi vengono scartati, le perdite sono 85

### Add followers DataFrame to db's new collection 'followers'

In [ ]:
Location = r'/home/simo/UnInfo/Tesi Magistrale/Dataset/elaborati/followers_df.csv'
df = pd.read_csv(Location)

desc_followers = df[df.columns[2]]
desc_user = df[df.columns[1]]


### test su campione di followers

In [ ]:
print len(desc_user)
print len(desc_followers)/10000

In [ ]:
# funzione di traduzione per gli username (non usata per ora)
def getId(username):
    user = db.user_translate.find( { 'username': username } )
    if user.count()>0 :
        return user[0]['id']
    else:
        return -1

In [ ]:
print getId('tolikng')

In [ ]:
def createFollowersJson(user, followed):
    # falls è la lista dei followers di uno user
    falls = json.loads(followed)

    # creo il json json_falls che conterrà lo username dello user in analisi, 
    # e l'array di json corrispondente agli utenti che segue.
    # creo l'array falls_array che è un array di json, ovvero gli utenti seguiti da quello user
    json_falls = {}
    
    # opzione per il caricamento delgi usernames tradotti
    #ID = getId(user)
    #if ID != -1:

    json_falls["username"] = user

    falls_array = []
    for f in falls:
        falls_array.append({'timestamp': f[0], 'follower_user': f[1]})

    json_falls["followers"] = falls_array


    #else:
        #return 0

    return json_falls

### insert followers on db

In [ ]:
%%time

db.drop_collection('followers')

for i in range(len(desc_followers)):

    json_elem = createFollowersJson(desc_user[i], desc_followers[i])
    if json_elem != 0:
        db.followers.insert_one(json_elem)


In [ ]:
cursor = db.followers.find()

print cursor.count()

cursor[0]

### Add likes DataFrame to db's new collection 'likes'

In [ ]:
Location = r'/home/simo/UnInfo/Tesi Magistrale/Dataset/elaborati/likes_df.csv'
df = pd.read_csv(Location)


### test su campione di likes

In [ ]:
desc_post = df[df.columns[2]]
desc_likes = df[df.columns[1]]

In [ ]:
print len(desc_post)/100000
print len(desc_likes)

print desc_post[2484374]
print desc_likes[2484374]

In [ ]:
def createLikesJson(post, likes):

    # falls è la lista dei followers di uno user
    like_list = json.loads(likes)

    # creo il json json_falls che conterrà lo username dello user in analisi, 
    # e l'array di json corrispondente agli utenti che segue.
    # creo l'array falls_array che è un array di json, ovvero gli utenti seguiti da quello user
    json_likes = {}
    json_likes["post"] = post

    likes_array = []

    for i in range(len(like_list)):
        #print like_list[i][0], ' - ', like_list[i][1], ' - ', like_list[i][2]
        likes_array.append({'timestamp': like_list[i][1], 'like_id': like_list[i][0], 'username': like_list[i][2]})

    json_likes["likes"] = likes_array


    return json_likes

In [ ]:
db.drop_collection('likes')

for i in range(len(desc_likes)):
    json_elem = createLikesJson(desc_post[i], desc_likes[i])
    
    db.likes.insert_one(json_elem)

In [ ]:
cursor = db.likes.find()
print cursor.count()

print cursor[2484374]


### Add shots DataFrame to db's new collection 'shots'

In [ ]:
Location = r'/home/simo/UnInfo/Tesi Magistrale/Dataset/shots.tsv'
df = pd.read_csv(Location, sep='\t')
desc_user = df[df.columns[0]]
desc_shots = df[df.columns[1]]

In [ ]:
desc[0]

In [ ]:
print len(desc_user)
print len(desc_shots)

In [ ]:
db.drop_collection('shots')

for i in range(len(desc_user)):

    json_shot_list = json.loads(desc_shots[i])
    
    json_shot = {}
    json_shot["username"] = desc_user[i]
    
    shots_array = []
    
    for s in json_shot_list:
        
        
        s.pop('attachments_count', None)
        s.pop('attachments_url', None)
        s.pop('buckets_count', None)
        s.pop('buckets_url', None)
        s.pop('comments_url', None)
        s.pop('html_url', None)
        s.pop('images', None)
        s.pop('normal', None)
        s.pop('teaser', None)
        s.pop('likes_url', None)
        s.pop('projects_url', None)
        s.pop('rebounds_count', None)
        s.pop('rebounds_url', None)
        


        try:
            d = datetime.strptime(s['created_at'], '%Y-%m-%dT%H:%M:%SZ')
            unixtime = time.mktime(d.timetuple())
            s['created_at'] = int(unixtime)

            d = datetime.strptime(s['updated_at'], '%Y-%m-%dT%H:%M:%SZ')
            unixtime = time.mktime(d.timetuple())
            s['updated_at'] = int(unixtime)
            
        except Exception as e: 
            print(e)

        shots_array.append(s)
        
        
            
    json_shot['shots'] = shots_array
    
    db.shots.insert_one(json_shot)
    
print 'Done!'

In [ ]:

cursor = db.shots.find()
print cursor.count()

In [ ]:
cursor[0]

In [ ]:
print cursor[771252]['username']
print len(cursor[771252]['shots'])

### metto sul db la tabella di traduzione id-username degli utenti

In [ ]:
cursor = db.users.find()
translate_table = pd.DataFrame()

username = []
user_id = []

for user in cursor:
    username.append(user['username'])
    user_id.append(user['id'])
    
translate_table.insert(loc=0, column='username', value=username, allow_duplicates=False)
translate_table.insert(loc=0, column='id', value=user_id, allow_duplicates=False)

In [ ]:

for i in range(translate_table.count()[0]):
    json_elem = {'username': translate_table['username'][i], 'id': translate_table['id'][i]}
    db.user_translate.insert_one(json_elem)

In [ ]:
cursor = db.user_translate.find()

In [ ]:
print cursor[0]

# Base di dati per gli shots , foramto singolo (non aggragati per utente)

In [ ]:
jsontemp = {'prova':1,'test':2}
print jsontemp

In [ ]:
db.shots.find().count()

In [ ]:
db.drop_collection('shots_single')

i=0

for elem in db.shots.find():
    for s in elem['shots']:
        s['user'] = elem['username']
        db.shots_single.insert_one(s)
        
    i=i+1
    print i


In [ ]:
db.shots_single.find().count()